In [1]:
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import random
import time
import re
import pandas as pd

In [ ]:
urls = ['https://volgograd.cian.ru/cat.php?deal_type=sale&engine_version=2&offer_type=flat&p={}&region=4704&room1=1',
       'https://volgograd.cian.ru/cat.php?deal_type=sale&engine_version=2&offer_type=flat&p={}&region=4704&room2=1',
       'https://volgograd.cian.ru/cat.php?deal_type=sale&engine_version=2&offer_type=flat&p={}&region=4704&room3=1]

        # 'https://volgograd.cian.ru/cat.php?deal_type=sale&engine_version=2&offer_type=flat&p={}&region=4704&room4=1&room5=1&room6=1'

In [ ]:
# Задерживаем выполнение кода
def wait_timer():
    secs = random.uniform(5,9)
    print(f'Programm is waiting for {secs} secs')
    time.sleep(secs)

In [ ]:
# Собираем ссылки на квартиры
def links_collecter(url):
    ua = UserAgent()
    header = {'User-Agent':str(ua.random)}
    
    clean_links = []
    
    wait_timer()
    response = requests.get(url, headers = header)
    if response.status_code != 200:
        print(f'There are some problems. The status code is {response.status_code}')
    else:
        print(f'Everything is good. The status code is {response.status_code}')
        response = BeautifulSoup(response.text, 'lxml')
        
        if response.title.text == 'Captcha - база объявлений ЦИАН':
            print(f'We got Captcha!')
            
        else:
            links = response.findAll('a', {'class':'_93444fe79c--link--39cNw'}) 
    
            for i in range(len(links)):
                clean_links.append(links[i]['href'])
            print(f'Another {len(clean_links)} links are ready')
    return clean_links

In [ ]:
# Все квартиры сразу: и вторичка, и новостройки. 1-2-3-4-5-6 комнат. Нет студий и нет свободных планировок
flats = [] # иначе сотру половину списка
all_flats = {} # информация о квартирах

In [ ]:
for i in range(1, 55):      # У нас 54 страницы на циане
    for url in urls:
        url = url.format(i)
        flats.extend(links_collecter(url))
        print(f'Page {i} was processed')

In [ ]:
# Далее необходимо собрать данные с каждой ссылки из flats
def information_taker(link):
    ua = UserAgent()
    header = {'User-Agent':str(ua.random)}
    
    wait_timer()
    response = requests.get(link, headers = header)
    
    if response.status_code != 200:
        print(f'There are some problems. The status code is {response.status_code}')
    else:
        print(f'Everything is good. The status code is {response.status_code}')
        response = BeautifulSoup(response.text, 'lxml')
        
        if response.title.text == 'Captcha - база объявлений ЦИАН':
            print(f'We got Captcha!')
            
        else:
            flat = {}
            flat['Ссылка'] = link   
            
            # Основная информация - площади, этаж
            inf_main_name = response.findAll('div', {'class':'a10a3f92e9--info-title--2bXM9'})
            inf_main = response.findAll('div', {'class':'a10a3f92e9--info-value--18c8R'})
            
            for i in range(0,len(inf_main)):
                flat[inf_main_name[i].text] = re.findall(r'\d+', inf_main[i].text)[0]
            
            # 
            inf_gen_name = response.findAll('span', {'class':'a10a3f92e9--name--3bt8k'})
            inf_gen = response.findAll('span', {'class':'a10a3f92e9--value--3Ftu5'})

            for i in range(0,len(inf_gen)):
                flat[inf_gen_n[i].text] = inf_gen[i].text

            inf_price = response.find('span', {'itemprop':'price'})
            flat['Цена'] =''.join(re.findall(r'\d+', inf_price.text))

            inf_room = response.find('h1', {'class':'a10a3f92e9--title--2Widg'})
            flat['Комнат'] = inf_room.text[0:7]

            inf_area = flat_page.findAll('a', {'class':'a10a3f92e9--link--1t8n1 a10a3f92e9--address-item--1clHr'})
            flat['Район'] = re.findall(r"р-н +\w+", str(inf_area))[0]
    
    return flat

In [ ]:
flats = set(flats) # Оставляем только уникальные ссылки на квартиры

In [ ]:
N = 0 # Номер квартиры
for link in flats:
    N = N + 1
    flat = information_taker(link)
    all_flats[N] = flat
    print('Flat number {} is done'.format(N))

In [ ]:
# Сохраняем собранные данные
df = pd.DataFrame(all_flats).T
df.to_csv('Cian.csv', encoding="utf-8")